In [ ]:
import os
import IPython
import matplotlib

% matplotlib notebook

# Gravity

## Motivation

In [ ]:
IPython.display.Image(url='https://anniebennettspain.files.wordpress.com/2012/06/img_1153.jpg')

## Constant translational acceleration in a straight line

Acceleration is defined as the rate of change of velocity:

$$a = \frac{\mathrm{d}v}{\mathrm{d}t}$$

Velocity (speed) is the rate of change of position:

$$v = \frac{\mathrm{d}x}{\mathrm{d}t}$$

In [ ]:
import sympy
from sympy import symbols, Eq, Integral, solve
from sympy.plotting import plot, plot_parametric, plot_implicit

sympy.init_printing()

In [ ]:
a, x, x0, v, v0, t, t0 = symbols('a x x_0 v v_0 t t_0')

In [ ]:
left = Integral(a, (t, 0, t))
right = Integral(1, (v, v0, v))
equation = Eq(left, right)
equation

In [ ]:
equation = equation.doit()
equation

In [ ]:
v = solve(equation, v)[0]
v

In [ ]:
left = Integral(v, (t, 0, t))
right = Integral(1, (x, x0, x))
equation = Eq(left, right)
equation

In [ ]:
equation = equation.doit()
equation

In [ ]:
solutions = solve(equation, t)
solutions

### Setting initial conditions and acceleration

In [ ]:
s0 = solutions[0].subs({'v_0': 0, 'x_0': 0, 'a': 9.8})
s0

In [ ]:
s1 = solutions[1].subs({'v_0': 0, 'x_0': 0, 'a': 9.8})
s1

In [ ]:
plot(s0, (x, 0, 10), title='Fall duration',
     xlabel='Distance (m)', ylabel='Time (s)')

In [ ]:
s0.subs('x', 13).evalf()

# Experiment

## Environment

TODO...

## Audio analysis

In [ ]:
import numpy
from scipy.signal import resample
from scipy.io import wavfile
from bokeh.plotting import figure, show
from bokeh.charts import Bar

from bokeh.io import output_notebook
output_notebook()

In [ ]:
wav = wavfile.read('data/gravity_audio/180.wav')
print(wav)

In [ ]:
DATA_PATH = os.path.join('data', 'gravity_audio')
samples = [wavfile.read(os.path.join(DATA_PATH, fname))[1]
           for fname in sorted(os.listdir(DATA_PATH))]

In [ ]:
def plot_audio(data):
    f = figure(width=800, height=400,
               title='WAV file plot',
               x_axis_label='Time (s)',
               y_axis_label='Amplitude')
    f.line(numpy.arange(len(data)) / 44100, numpy.array(data))
    show(f)

In [ ]:
plot_audio(samples[0])

In [ ]:
plot_audio(samples[-1])

## Convolution

In [ ]:
WINDOW = int(0.02 * len(samples[-1]))

In [ ]:
def convolution(data, window):
    filtered = [0] * len(data)
    for i in range(window - 1, len(data)):
        for j in range(window):
            filtered[i] = max(filtered[i], abs(data[i - j]))
    return filtered

In [ ]:
%time filtered = convolution(samples[-1], WINDOW)

In [ ]:
plot_audio(filtered)

### Optimization

In [ ]:
import numba

@numba.jit
def convolution(data, window):
    filtered = [0] * len(data)
    for i in range(window - 1, len(data)):
        for j in range(window):
            filtered[i] = max(filtered[i], abs(data[i - j]))
    return filtered

In [ ]:
%timeit filtered = convolution(samples[-1], WINDOW)

In [ ]:
@numba.jit
def convolution(data, window):
    filtered = numpy.zeros(len(data))
    abs_data = numpy.abs(data)
    for i in range(window - 1, len(data)):
        filtered[i] = abs_data[i - window:i].max()
    return filtered

In [ ]:
%timeit filtered_numba = convolution(samples[-1], WINDOW)

In [ ]:
import pandas

In [ ]:
%timeit filtered = pandas.rolling_max(numpy.abs(samples[-1]), WINDOW)

## Edge detection

In [ ]:
@numba.jit
def edge_detect(data, threshold=30000):
    filtered = numpy.zeros(len(data))
    N = int(0.02 * len(data))
    for i in range(N - 1, len(data)):
        if data[i] <= threshold:
            continue
        filtered[i] = 1.
        for j in range(N - 1):
            if data[i - j - 1] > threshold:
                filtered[i] = 0.
                break
    return filtered

In [ ]:
edge = edge_detect(filtered)
edge.sum()

In [ ]:
plot_audio(edge)

In [ ]:
itemindex = numpy.where(edge==1)
time_diff = (itemindex[0][1] - itemindex[0][0]) / 44100
time_diff

## Time

In [ ]:
samples = [wavfile.read(os.path.join(DATA_PATH, fname))[1]
           for fname in sorted(os.listdir(DATA_PATH))]
times = [0]
for data in samples:
    filtered = convolution(data, WINDOW)
    edge = edge_detect(filtered)
    itemindex = numpy.where(edge==1)
    time_diff = (itemindex[0][1] - itemindex[0][0]) / 44100
    times.append(time_diff)
times = numpy.array(times)

In [ ]:
distances = numpy.arange(len(times)) * 0.05
g_estimations = (2 * distances) / (times ** 2)

g_estimated = g_estimations.mean()
error = (9.8 - g_estimated) / 9.8

print(g_estimated, error)

In [ ]:
p = Bar([9.8, g_estimated], width=400, height=400)
show(p)

In [ ]:
real_times = [numpy.sqrt((2 * 0.05 * i) / 9.8) for i in range(len(times))]


f = figure(width=800, height=400,
           title='Theoretical versus measured time',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(distances, times, fill_color=None, color='red', legend='Measured')
f.circle(distances, real_times, fill_color=None, legend='Theoretical')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
# TODO (could the air be the cause of the increasingly growing error?)

We can calculate the maximum absolute error in the time measure (in seconds):

In [ ]:
numpy.abs(real_times - times).max()

In [ ]:
error = times - real_times

f = figure(width=800, height=400,
           title='Measured time error',
           x_axis_label='Distance(m)',
           y_axis_label='Error (ms)')
f.line(distances, error)
show(f)

# TODO

- Implement Python-based linear regression?
- Show simple equations

# Linear regression

In [ ]:
df = pandas.DataFrame({'distances': distances, 'times': times})
df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(df[['distances']], df['times'])
print(model.intercept_, model.coef_)

In [ ]:
predictions = model.predict(df[['distances']])
predictions

In [ ]:
plt.figure()
plt.scatter(df['distances'], df['times'],  color='black')
plt.plot(df['distances'], predictions, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
rss = np.sum((predictions - df['times']) ** 2)
t_prediction = model.predict(13)[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %s' % rss)
print('Time (pred): %s' % t_prediction)
print('Time (real): %s' % t_real)
print('Error: %s' % (t_prediction - t_real))

In [ ]:
model.predict(13)

In [ ]:
s0.subs('x', 13).evalf()

## Linear regression (new features)

In [ ]:
df['sqrt_distances'] = numpy.sqrt(df['distances'])
df.head()

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(df[['sqrt_distances']], df['times'])

predictions = model.predict(df[['sqrt_distances']])

plt.figure()
plt.scatter(df['distances'], df['times'],  color='black')
plt.plot(df['distances'], predictions, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
rss = np.sum((predictions - df['times']) ** 2)
t_prediction = model.predict(numpy.sqrt(13))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %s' % rss)
print('Time (pred): %s' % t_prediction)
print('Time (real): %s' % t_real)
print('Error: %s' % (t_prediction - t_real))

## Linear regression (multiple features)

In [ ]:
# TODO

- Regression (line, sqrt, multiple)
- Prediction capabilities (in and out of range)
- Regularization (L1 vs. L2)
- Compare predictions (slider?) http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#userguide-interaction